In [2]:
# A few helper functions for posting and getting api requests
import json
import requests
import pandas
import time

base_url = 'http://biggim.ncats.io/api'

#a couple of simple helper functions
def post(endpoint, data={}, base_url=base_url):
    req = requests.post('%s/%s' % (base_url,endpoint), data=data)
    req.raise_for_status()
    return req.json()

def get(endpoint, data={}, base_url=base_url):
    req = requests.get('%s/%s' % (base_url,endpoint), data=data)
    try:
        req.raise_for_status()
    except requests.HTTPError as e:
        print("Sent: GET %s?%s" % (req.request.url,req.request.body))
        if e.response.status_code == 400:
            print(e.response)
            jprint(e.response.json())
        raise
    print("Sent: GET %s?%s" % (req.request.url,req.request.body))
    return req.json()
    

def jprint(dct):
    print(json.dumps(dct, indent=2))
    
def wrapper(endpoint, data={}, base_url=base_url):
    try:
        response = get(endpoint, data, base_url)
        jprint(response)
    except requests.HTTPError as e:

        print(e)
        if e.response.status_code == 400:
            jprint(e.response.json())
        raise
    try:
        ctr = 1
        while True:
            query_status = get('%s/status/%s'% (endpoint.split('/')[0],response['request_id'],))
            jprint(query_status)
            if query_status['status'] !='running':
                # query has finished
                break
            else:
                time.sleep(ctr)
                ctr += 5
                #linear backoff
                print("Checking again")
    except requests.HTTPError as e:
        print(e)
        if e.response.status_code == 400:
            jprint(e.response.json())
        raise
    return pandas.concat(map(pandas.read_csv, query_status['request_uri']))

    

In [3]:

import requests
def query_single_edge(_input, _output, _value):
    """
        Retrieve results using one API call from input to output.
        :param _input: the input prefix, for a list of input prefix in BioThings Explorer, visit: http://biothings.io/explorer/api/v2/metadata/bioentities
        :param _output: the output prefix, for a list of output prefix in BioThings Explorer, visit: http://biothings.io/explorer/api/v2/metadata/bioentities
        :_value: The value of the input
        :return:
    """
    """
    endpoint = 'directinput2output'
    base_url = 'http://biothings.io/explorer/api/v2'
    data = {'output_prefix':_output,
            'input_prefix': _input,
            'input_value': _value,
            'format': 'translator'
    }"""
    #doc = get(endpoint, data=data, base_url=base_url)
    qurl = 'http://biothings.io/explorer/api/v2/directinput2output?'
    qurl += 'input_prefix=%s&output_prefix=%s&input_value=%s&format=translator'
    qurl = qurl % (_input, _output, _value)
    print(qurl)
    try:
        req = requests.get(qurl.strip(), timeout=10)
    except:
        print("timeout")
        return []
    #print("Sent: GET %s?%s" % (req.request.url,req.request.body))
    return req.json()
_="""
phenotypes = ['HP:0002204',
 'HP:0000823',
 'HP:0000980',
 'HP:0001941',
 'HP:0004825',
 'HP:0000097',
 'HP:0007917',
 'HP:0003155',
 'HP:0001712']
similar_conditions = []

# loop through the first 100 human phenotype ontology IDs got from CELL 12
for _phenotype in phenotypes[:100]:
    results = query_single_edge(_input='hp', 
                                _output='omim.disease', 
                                _value=_phenotype.split(':')[-1])
    if results and 'result_list' in results:
        similar_conditions += results['result_list']['node_list']
# extract the unique conditions from results
similar_conditions = set([_doc['id'] for _doc in similar_conditions])
implicated_genes = []
print("HERE2")
# loop through the first 100 similar conditions got from CELL 15
for _condition in list(similar_conditions)[:100]:
    results = query_single_edge(_input='omim.disease', 
                                _output='ncbigene', 
                                _value=_condition.split(':')[-1])
    if results and 'result_list' in results:
        implicated_genes += results['result_list']['node_list']
print("HERE3")"""

In [4]:
from multiprocessing import Pool
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
#my_pool = Pool(1)

#result = query_single_edge(_input='doid', 
#                                _output='hp', 
#                                _value='6000')
def el(result):
    try:
        return result['result_list']['edge_list']
    except:
        logging.warning(result)
        return []
    

def _doid_to_hp(doid):

    
    qse = query_single_edge(_input='doid', 
                                _output='hp', 
                                _value=doid
                           )
    results = el(qse)
    ts = set([x['target_id'] for x in results])
    return [x.split(':')[-1] for x in ts if x]

def doid_to_hp(doids, limit=None):
    
    #res = my_pool.map(_doid_to_hp, doids)
    res = []
    
    for doid in doids:
        if limit is None or len(set(res)) < limit:
            res += _doid_to_hp(doid)
    return res



def hp_to_omim(hpids, limit=None):
    #res = my_pool.map(_hp_to_omim, hpids)
    res = []
    for hpid in hpids:
        if limit is None or len(set(res)) < limit:
            res += _hp_to_omim(hpid)
    return res

def _hp_to_omim(hpid):
    qse = query_single_edge(_input='hp', 
                                _output='omim.disease', 
                                _value=hpid)
    results = el(qse)
    ts = set([x['target_id'] for x in results])
    return [x.split(':')[-1] for x in ts if x]

def omim_to_gene(omims, limit=None):
    #res = my_pool.map(_omim_to_gene, omims)
    res = []
    for omim in omims:
        if limit is None or len(set(res)) > limit:
            res += _omim_to_gene(omim)
    return res

def _omim_to_gene(omim):
    qse = query_single_edge(_input='omim.disease', 
                                _output='ncbigene',  
                                _value=omim)
    results = el(qse)
    ts = set([x['target_id'] for x in results])
    return [x.split(':')[-1] for x in ts if x]

def _hp_to_uberon(hpid):

    qse = query_single_edge(_input='hp', 
                                _output='uberon', 
                                _value=hpid)
    
    results = el(qse)
    ts = set([x['target_id'] for x in results])
    return [x.split(':')[-1] for x in ts if x]

def hp_to_uberon(hpids, limit=None):
    #res = my_pool.map(_hp_to_uberon,hpids)
    res = []
    
    for hpid in hpids:
        if limit is None or len(set(res)) > limit:
            res += _hp_to_uberon(hpid)
    return res



def uberon_to_bto(ubid):
    uberon_to_bto = json.load(open('bto_uberon_bg.json'))
    for x in uberon_to_bto:
        if x['uberon_id'] == ubid and x['bg_label']:
            return x['bg_label']
    return None



Metadata
DOID	DOID:678
Name	progressive supranuclear palsy
Definition	A movement disease characterized by serious and progressive problems with control of gait and balance, along with complex eye movement and thinking problems. It involves gradual deterioration and death of specific volumes of the brain. 
http://en.wikipedia.org/wiki/Progressive_supranuclear_palsy, http://www.ninds.nih.gov/disorders/psp/psp.htm
Xrefs	
GARD:7471

ICD10CM:G23.1

MESH:D013494

MTHICD9_2006:333.0

NCI:C85028

OMIM:601104

OMIM:609454

OMIM:610898

ORDO:683

SNOMEDCT_US_2018_03_01:192975003

SNOMEDCT_US_2018_03_01:192976002

SNOMEDCT_US_2018_03_01:28978003

UMLS_CUI:C0038868

Subsets	
DO_rare_slim

NCIthesaurus

Synonyms	
progressive supranuclear ophthalmoplegia [EXACT]

Steele-Richardson-Olszewski syndrome [EXACT]

Relationships	
is_a movement disease

Add an item to the term tracker


In [5]:
#http://disease-ontology.org/
disease_doid = ["678"]
hpids = doid_to_hp(disease_doid)
if len(hpids) == 0:
    raise Exception("No phenotypes related to %s" % (str(disease_doid)))
else:
    print("Returned %i phenotypes" % (len(hpids), ))

http://biothings.io/explorer/api/v2/directinput2output?input_prefix=doid&output_prefix=hp&input_value=678&format=translator
Returned 26 phenotypes


In [6]:
#https://hpo.jax.org/app/
omims = hp_to_omim(hpids, limit=50)

if len(omims) == 0:
    raise Exception("No mendelian diseases(OMIM) related to %s" % (str(hpids)))
else:
    print("Returned %i mendelian diseases" % (len(hpids), ))

http://biothings.io/explorer/api/v2/directinput2output?input_prefix=hp&output_prefix=omim.disease&input_value=0001337&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=hp&output_prefix=omim.disease&input_value=0000623&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=hp&output_prefix=omim.disease&input_value=0012535&format=translator


http://biothings.io/explorer/api/v2/directinput2output?input_prefix=hp&output_prefix=omim.disease&input_value=0100543&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=hp&output_prefix=omim.disease&input_value=0002063&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=hp&output_prefix=omim.disease&input_value=0000496&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=hp&output_prefix=omim.disease&input_value=0000511&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=hp&output_prefix=omim.disease&input_value=0001332&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=hp&output_prefix=omim.disease&input_value=0002172&format=translator
Returned 26 mendelian diseases


In [7]:
#https://omim.org/
genes = omim_to_gene(omims)
#genes are ncbi
if len(genes) == 0:
    raise Exception("No genes associated with %s" % (str(omims),))
else:
    print("Returned %i genes" % (len(omims,)))

http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=615400&format=translator


http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=607876&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=616187&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=611808&format=translator


http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=168100&format=translator


http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=616421&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=616494&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=610297&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=164500&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=109150&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=231005&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=183090&format=translator
http://biothi

http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=606324&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=168100&format=translator


...

http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=224550&format=translator


http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=614932&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=607488&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=105300&format=translator


http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=611031&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=611092&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=612390&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=607060&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=260300&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=615530&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=613855&format=translator
http://biothi

http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=616410&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=614203&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=605909&format=translator
http://biothings.io/explorer/api/v2/directinput2output?input_prefix=omim.disease&output_prefix=ncbigene&input_value=607688&format=translator


Returned 60 genes


In [8]:
gene_list = ','.join(genes)


In [9]:
res = get('/metadata/tissue/brain')

Sent: GET http://biggim.ncats.io/api//metadata/tissue/brain?None


In [10]:
for s in res['substudies']:
    for ss in  s['columns']:
        if 'GTEx' in ss['name'] :
            print(ss['name'])

GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Pituitary_Correlation
GTEx_Pituitary_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GTEx_Brain_Correlation
GTEx_Brain_Pvalue
GT

In [16]:
columns = ["GTEx_Brain_Correlation"]
example_query = {
  "restriction_gt": ','.join(["%s,0.5" % (c,) for c in columns]),
  "table": "BigGIM_70_v1",
  "columns": ','.join(columns),
  "ids1": gene_list,
  "limit": 100000
}
try:
    query_submit = get('biggim/query', data=example_query)
    jprint(query_submit)
except requests.HTTPError as e:
    print(e)
    
    jprint(e.response.json())


try:
    while True:
        query_status = get('biggim/status/%s'% (query_submit['request_id'],))
        jprint(query_status)
        if query_status['status'] !='running':
            # query has finished
            break
        else:
            time.sleep(1)
            print("Checking again")
except requests.HTTPError as e:
    print(e)
    
    jprint(e.response.json())
    
result = pandas.concat(map(pandas.read_csv, query_status['request_uri']))
result

Sent: GET http://biggim.ncats.io/api/biggim/query?ids1=104&ids1=119&ids1=160&ids1=161&ids1=273&ids1=478&ids1=491&ids1=492&ids1=498&ids1=506&ids1=523&ids1=526&ids1=528&ids1=534&ids1=535&ids1=622&ids1=773&ids1=774&ids1=785&ids1=790&ids1=808&ids1=814&ids1=863&ids1=899&ids1=1016&ids1=1114&ids1=1121&ids1=1159&ids1=1196&ids1=1213&ids1=1431&ids1=1609&ids1=1639&ids1=1653&ids1=1665&ids1=1759&ids1=1778&ids1=1780&ids1=1795&ids1=1801&ids1=1856&ids1=1859&ids1=1917&ids1=1951&ids1=1974&ids1=2026&ids1=2175&ids1=2254&ids1=2554&ids1=2561&ids1=2562&ids1=2563&ids1=2566&ids1=2583&ids1=2664&ids1=2686&ids1=2743&ids1=2744&ids1=2805&ids1=2821&ids1=2835&ids1=2845&ids1=2898&ids1=2911&ids1=2931&ids1=3052&ids1=3064&ids1=3093&ids1=3098&ids1=3376&ids1=3508&ids1=3607&ids1=3631&ids1=3667&ids1=3746&ids1=3748&ids1=3760&ids1=3778&ids1=3845&ids1=3897&ids1=3988&ids1=4137&ids1=4139&ids1=4152&ids1=4185&ids1=4209&ids1=4221&ids1=4293&ids1=4591&ids1=4628&ids1=4644&ids1=4665&ids1=4674&ids1=4905&ids1=4926&ids1=4943&ids1=4952&ids1

,GPID,Gene1,Gene2,GTEx_Brain_Correlation
0,140000000104,104,14,0.7680
1,160000000104,104,16,0.7110
2,210000000104,104,21,0.8469
3,230000000104,104,23,0.7666
4,270000000104,104,27,0.7517
5,360000000104,104,36,0.7890
6,400000000104,104,40,0.6795
7,410000000104,104,41,0.7086
8,430000000104,104,43,0.7152
9,480000000104,104,48,0.5788


In [17]:

df  =  result
df  = df.sort_values(by=columns[0],ascending=False)
df = df.reset_index()
del df['index']
gene_list = []
i = len(gene_list)
for index, row in df.iterrows():
    if i==500:
        break
    gene_list.append(row['Gene1'])
    gene_list.append(row['Gene2'])
    gene_list = list(set(gene_list))
    i = len(gene_list)

gene_list.sort()
gene_list = [int(x) for x in gene_list]
gene_list = [str(x) for x in gene_list]
#gene_list = gene_list.sort(key=str)
str1 = ','.join(gene_list)
print(str1)

104,119,161,211,273,287,291,440,476,491,492,506,534,535,605,673,773,774,776,783,785,862,863,891,898,1005,1016,1114,1159,1310,1431,1487,1609,1795,1801,1804,1859,1951,2026,2071,2175,2254,2259,2475,2561,2583,2686,2744,2821,2822,2898,2911,2932,2941,2962,3035,3093,3097,3098,3361,3382,3420,3508,3631,3667,3675,3746,3748,3751,3752,3767,3768,3795,3897,3899,4137,4141,4204,4209,4221,4293,4524,4591,4644,4705,4943,4978,5001,5048,5127,5213,5291,5310,5463,5526,5528,5532,5563,5590,5601,5664,5709,5775,5801,6014,6252,6257,6262,6326,6327,6334,6456,6543,6586,6597,6616,6709,6710,6712,6732,6785,6812,6860,6904,7014,7204,7329,7342,7464,7504,7644,7701,7732,7786,7874,8120,8208,8395,8480,8497,8506,8526,8567,8604,8720,8729,8775,8825,8836,8841,8874,8925,8927,8943,9024,9025,9063,9110,9145,9331,9344,9369,9372,9379,9382,9470,9475,9515,9520,9575,9671,9677,9690,9701,9737,9746,9762,9764,9777,9806,9807,9820,9826,9847,9853,9862,9892,9900,9912,9980,10038,10057,10059,10128,10129,10147,10193,10243,10296,10367,10369,10370,104

In [18]:
example_query = {
  "restriction_gt": ','.join(["%s,0.5" % (c,) for c in columns]),
  "table": "BigGIM_v1",
  "columns": ','.join(columns),
  "ids1": str1,
  "ids2": str1,
  "limit": 100000000000
}
try:
    query_submit = get('biggim/query', data=example_query)
    jprint(query_submit)
except requests.HTTPError as e:
    print(e)
    
    jprint(e.response.json())


try:
    while True:
        query_status = get('biggim/status/%s'% (query_submit['request_id'],))
        jprint(query_status)
        if query_status['status'] !='running':
            # query has finished
            break
        else:
            time.sleep(1)
            print("Checking again")
except requests.HTTPError as e:
    print(e)
    
    jprint(e.response.json())
    
result = pandas.concat(map(pandas.read_csv, query_status['request_uri']))
result

Sent: GET http://biggim.ncats.io/api/biggim/query?table=BigGIM_v1&limit=100000000000&columns=GTEx_Brain_Correlation&restriction_gt=GTEx_Brain_Correlation%2C0.5&ids2=104%2C119%2C161%2C211%2C273%2C287%2C291%2C440%2C476%2C491%2C492%2C506%2C534%2C535%2C605%2C673%2C773%2C774%2C776%2C783%2C785%2C862%2C863%2C891%2C898%2C1005%2C1016%2C1114%2C1159%2C1310%2C1431%2C1487%2C1609%2C1795%2C1801%2C1804%2C1859%2C1951%2C2026%2C2071%2C2175%2C2254%2C2259%2C2475%2C2561%2C2583%2C2686%2C2744%2C2821%2C2822%2C2898%2C2911%2C2932%2C2941%2C2962%2C3035%2C3093%2C3097%2C3098%2C3361%2C3382%2C3420%2C3508%2C3631%2C3667%2C3675%2C3746%2C3748%2C3751%2C3752%2C3767%2C3768%2C3795%2C3897%2C3899%2C4137%2C4141%2C4204%2C4209%2C4221%2C4293%2C4524%2C4591%2C4644%2C4705%2C4943%2C4978%2C5001%2C5048%2C5127%2C5213%2C5291%2C5310%2C5463%2C5526%2C5528%2C5532%2C5563%2C5590%2C5601%2C5664%2C5709%2C5775%2C5801%2C6014%2C6252%2C6257%2C6262%2C6326%2C6327%2C6334%2C6456%2C6543%2C6586%2C6597%2C6616%2C6709%2C6710%2C6712%2C6732%2C6785%2C6812%2C6860%2

,GPID,Gene1,Gene2,GTEx_Brain_Correlation
0,55320000284339,284339,5532,0.6875
1,88360000009762,9762,8836,0.7500
2,37670000051028,51028,3767,0.7500
3,7850000054946,54946,785,0.7500
4,1225530000283373,283373,122553,0.7500
5,22590000002561,2561,2259,0.8125
6,259770000079369,79369,25977,0.8125
7,236440000051209,51209,23644,0.8125
8,70140000008720,8720,7014,0.8125
9,8620000007701,7701,862,0.8125
